<a href="https://colab.research.google.com/github/samitf/Machine-Learning/blob/main/BikeShareDemand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Problem Statement:**

We want to predict how many bikes people will rent at a specific time using details like weather, season, and time. This helps in planning and making sure enough bikes are available when needed.

# **Problem Definition:**

The goal is to build a predictive model that estimates the total number of bikes rented at a given hour or day. The model will use input features such as:

*   Time-based features: hour, day, month, weekday, year
*   Environmental features: temperature, humidity, windspeed, weather situation
*   Calendar features: season, holiday, working day

This is a regression problem where the target variable is the continuous value cnt, which represents the total number of bikes rented (both casual and registered users). The model should learn patterns in the historical data to make accurate predictions for future rental demand.

In this project, We develop a machine learning model to predict daily bike rentals for a city bike-sharing program. This information can be used for optimizing bike distribution, staffing, and overall program efficiency.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
PATH = "/content"

In [ ]:
df_raw = pd.read_csv(f'{PATH}/hour.csv')
print(df_raw.head())